In [ ]:
!pip install Datasets transformers --quiet

In [ ]:
!pip install torch --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 59.9 MB/s eta 0:00:00


In [ ]:
!pip install evaluate --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.4 MB/s eta 0:00:00


In [ ]:
!pip install rouge_score --quiet

  Preparing metadata (setup.py) ... done


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, pipeline
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import evaluate
from sentence_transformers import SentenceTransformer, util
import torch
import re

In [ ]:
dt = load_dataset('SQuAD_v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [ ]:
dt

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})

In [ ]:
dt['train'][0]

{'id': '56be85543aeaaa14008c9063',
 'title': 'Beyoncé',
 'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".',
 'question': 'When did Beyonce start becoming popular?',
 'answers': {'text': ['in the late 1990s'], 'answer_start': [269]}}

In [ ]:
f100_context=[]
f100_question=[]
f100_answer=[]
for example in dt['train'].shuffle(seed = 42).select(range(100)):
  f100_context.append(example['context'])
  f100_question.append(example['question'])
  f100_answer.append(example['answers'])

In [ ]:
generator = pipeline("text-generation", model='gpt2')
generated_answers= []
for question in f100_question:
  prompt = f"Question: {question} Answer:"
  result = generator(prompt, max_length= 100)
  generated_answers.append(result[0]['generated_text'])



config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transform

In [ ]:
generated_answers[0]

"Question: What term characterizes the intersection of the rites with the Roman Catholic Church? Answer: It is a bit like the intersection of the rites with the Roman Catholic Church. As such, we do not take into consideration the intertextual aspects of the rites. We take into account the Catholic character, the Church's character, the priestly function of her, the life of her and her.\n\nNow, I'm not saying that there is nothing wrong with Catholic ritual. I'm saying that the Catholic Church has a lot of character. But the church's character is not limited to the Roman Catholic Church. The Church has a lot of other aspects to it.\n\nIn the Roman Catholic Church, the priest is not a person who is the mother of sin. He is not an individual, but a person who is a person of God. A priest is one who is not the mother of sin. He is not the priest who is the mother of death, but the one who is the father of death. A priest is the one who is the father of death.\n\nBut the priest is a person

In [ ]:
def clean_answer(text):
    if "Answer:" in text:
        text = text.split("Answer:")[-1]
    text = re.sub(r"Question:.*", "", text)
    text = text.strip()
    if "." in text:
        text = text.split(".")[0] + "."
    return text.strip()

cleaned_generated_answers = [clean_answer(ans) for ans in generated_answers]


In [ ]:
cleaned_generated_answers[:5]

['It is a bit like the intersection of the rites with the Roman Catholic Church.',
 'In the 8th century,',
 'That',
 'A number of key foods are important for our people, such as meat, dairy and poultry.',
 'Because it is the first flute made by the king, and it is extremely difficult to']

In [ ]:
cleaned_gold_answers = []
for item in f100_answer:
    answers = item['text']
    if not answers:
        cleaned_gold_answers.append([''])
    else:
        cleaned = [clean_answer(ans) for ans in answers]
        cleaned_gold_answers.append(cleaned)


In [ ]:
cleaned_gold_answers[:5]

[['full union'],
 ['important trade center'],
 ['disparaging individual decision-making'],
 ['black bread, pork, potatoes, and dairy products.'],
 ['wood']]

In [ ]:
rouge_metric = evaluate.load("rouge")

In [ ]:
rouge_score = rouge_metric.compute(predictions=cleaned_generated_answers,references=cleaned_gold_answers)
rouge_score

{'rouge1': np.float64(0.037449206442904406),
 'rouge2': np.float64(0.0024215686274509803),
 'rougeL': np.float64(0.03174687940798024),
 'rougeLsum': np.float64(0.03135866910866911)}

In [ ]:
# Let's do better using RAG and sentence transformer
model = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
encoded_context = model.encode(f100_context, show_progress_bar=True)

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [ ]:
encoded_question= model.encode(f100_question, show_progress_bar=True)

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [ ]:
k= 3
retrieved_contexts_per_question = []

for question in encoded_question:

    sim_matrix = util.cos_sim(question, encoded_context)[0]
    top_results = torch.topk(sim_matrix, k=k)
    top_indices = top_results.indices.tolist()

    top_contexts = [f100_context[i] for i in top_indices]
    retrieved_contexts_per_question.append(top_contexts)

In [ ]:
retrieved_contexts_per_question[0]

['The Roman Catholic Church canon law also includes the main five rites (groups) of churches which are in full union with the Roman Catholic Church and the Supreme Pontiff:',
 "Predominantly Christian, the population of St. John's was once divided along sectarian (Catholic/Protestant) lines. In recent years, this sectarianism has declined significantly, and is no longer a commonly acknowledged facet of life in St. John's. St. John's is the seat of the Roman Catholic Archbishop of St. John's, and the Anglican Bishop of Eastern Newfoundland and Labrador. All major Christian sects showed a decline from 2001–2011 with a large increase in those with no religion from 3.9% to 11.1%.",
 'A German ethnicity emerged in the course of the Middle Ages, ultimately as a result of the formation of the kingdom of Germany within East Francia and later the Holy Roman Empire, beginning in the 9th century. The process was gradual and lacked any clear definition, and the use of exonyms designating "the Germ

In [ ]:
rag_prompts = []

for i in range(len(f100_question)):
    top_contexts = retrieved_contexts_per_question[i]
    question = f100_question[i]

    context_block = "\n\n".join(top_contexts)

    prompt = f"{context_block}\n\nQuestion: {question} Answer:"
    rag_prompts.append(prompt)


In [ ]:
rag_generated_answers = []

for prompt in rag_prompts:
    result = generator(prompt, max_length=50)
    rag_generated_answers.append(result[0]['generated_text'])


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_

In [ ]:
cleaned_RAG_answers = [clean_answer(ans) for ans in rag_generated_answers]


In [ ]:
rag_rouge = rouge_metric.compute(predictions= cleaned_RAG_answers, references= cleaned_gold_answers)
rag_rouge

{'rouge1': np.float64(0.04080086906696065),
 'rouge2': np.float64(0.004822032909324309),
 'rougeL': np.float64(0.036275902782981075),
 'rougeLsum': np.float64(0.03532337844760446)}